<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Case-List-Refresh-Notebook" data-toc-modified-id="Case-List-Refresh-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Case List Refresh Notebook</a></span><ul class="toc-item"><li><span><a href="#TODO" data-toc-modified-id="TODO-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>TODO</a></span></li></ul></li><li><span><a href="#BREAK" data-toc-modified-id="BREAK-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>BREAK</a></span></li><li><span><a href="#Walmart" data-toc-modified-id="Walmart-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Walmart</a></span></li></ul></div>

# Case List Refresh Notebook

## TODO

- Ask CSM about appropriate columns
- Transfer knowledge to working framework
- Work on WM portion
    - Detail what columns to pull from SQL
    - Create functionalities
- Generalize functionalities for any network


Thought process: Have the same outputs for both SC and WM and eventually Albertsons. This should be refreshed at least once a week, hopefully done by Jenkins or another chron tool. It should just run as a script rather than a program, though functionality should be given to do both. 

In [50]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def mysql_query(query='DEFAULT', verbose=False):
    import sqlalchemy as sql
    username = 'jyu'
    password = 't4pu6["Hw+X@=~gL'
    connection = '192.168.157.106'
    database_name = 'ods'
    connect_string = f'mysql://{username}:{password}@{connection}/{database_name}'
    sql_engine = sql.create_engine(connect_string)
    df = pd.read_sql_query(query, sql_engine)
    if verbose: print(df.shape)
    return df

In [73]:
def gsheet_import_caselist(network, verbose=False):
    # Relating network to sheet index
    network_dict = {
        'Sams Club':0,
        'Walmart':1
    }
    network_choice = network_dict[network]
    
    import pygsheets
    gc = pygsheets.authorize(client_secret='client_secret.json')
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wsnBd3AHObl4gnUJ2MlwkusuXRoOsQpu2Kx6zGH8bVY/edit#gid=0')
    df = sh[network_choice].get_as_df()
    
    if verbose: print(df.shape)
    
    print(f'Successfully returned {network}.')
    
    return df

def SC_metrics_pull():
    query = """
    SELECT metrics.program, metrics.weeks_post, metrics.start_date, metrics.end_date, 
    metrics.item_number, metrics.item_description, metrics.category, metrics.subcategory,
    metrics.category_num, metrics.subcat_num, metrics.Freeosk_clubs, metrics.control_clubs,
    metrics.transactions, metrics.scans, metrics.audience, metrics.engagement, metrics.conversion,
    metrics.repeat, metrics.Immediate, metrics.`HH_A%%`, metrics.`HH_B%%`, metrics.`HH_C%%`,
    lift.freeosk_lift, lift.control_lift, cfg.merchandising_type, cfg.placement_type
    FROM longitudinal.c_metrics as metrics
    JOIN longitudinal.c_lift as lift ON metrics.Program = lift.program_name
    JOIN longitudinal.c_cfg as cfg on metrics.program = cfg.program_name
    WHERE metrics.Weeks_post = '12W';
    """
    raw_sc_df = mysql_query(query)
    return raw_sc_df

def SC_formatter(raw_sc_df=SC_metrics_pull()):
    # Formatting
    print(f'Successfully returned the MySQL query')
    sc_df = raw_sc_df.copy()
    sc_df['Traffic/club'] = sc_df['audience'] / sc_df['Freeosk_clubs']
    sc_df['Scans/club'] = sc_df['scans'] / sc_df['Freeosk_clubs']
    sc_df['Program2'] = sc_df['program'].str.split("_", n=1, expand=True)[0]
    sc_df['Unique Identifier'] = sc_df['Program2'] + sc_df['item_number'].astype(str)
    renamed_columns = {
        'HH_A%': 'A%',
        'HH_B%': 'B%',
        'HH_C%': 'C%'
    }
    sc_df = sc_df.rename(columns=renamed_columns)
    sc_df.loc[:, 'Tags'] = ''
    sc_df.loc[:, 'Notes'] = ''
    sc_df.columns = [column.replace('_', ' ') for column in list(sc_df)]
    sc_df.columns = [column.capitalize() for column in list(sc_df)]
    print(f'Successfully formatted the MySQL query')
    
    # Mapping to food boolean
    food_mapping = pd.read_excel('Food_Mapping.xlsx')
    sc_df_2 = pd.merge(sc_df, food_mapping,  how='left', 
                       left_on=['Category num','Subcat num'], 
                       right_on = ['CATEGORY_NUMBER','SUB_CATEGORY_NUMBER'])
    print(f'Successfully joined to Food_Mapping.xslx')
    
    # Removing duplicates
    sc_df_3 = sc_df_2[sc_df_2['Program'].str.contains('_12.zip')]
    
    # Final column rearrangement
    rearranged_columns = ['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 
                      'End date', 'Item number', 'Item description', 'Category', 
                      'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 
                      'Control clubs', 'Transactions', 'Audience', 'Traffic/club', 
                      'Scans', 'Scans/club', 'Engagement', 'Conversion', 'Repeat', 
                      'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 
                      'Merchandising type', 'Placement type', 'Program2', 'Unique identifier', 'Food']
    
    sc_df_3 = sc_df_3[rearranged_columns].reset_index(drop=True)
    
    print(f'SC_Formatter completed! Shape is {sc_df_3.shape}')
    
    return sc_df_3

def sc_gsheet_appender(sc_df):
    
    gsheet_import_sc_df = gsheet_import_caselist('Sams Club')
    
    print(f'Sams Club Dataframe has (rows, columns): {sc_df.shape}')
    print(f'Imported Gsheet Dataframe has (rows, columns): {gsheet_import_sc_df.shape}')
    
    # Checks for rows to append without edits to previous rows
    sc_df_programlist = sc_df['Program'].to_list()
    gsheet_import_programlist = gsheet_import_sc_df['Program'].to_list()
    to_append = [x for x in sc_df_programlist if x not in gsheet_import_programlist]
    print('Rows to append: ' + str(len(to_append)))
    
    gsheet_import_sc_df = gsheet_import_sc_df.set_index('Program')
    sc_df = sc_df.set_index('Program')
    
    gsheet_import_sc_appended = gsheet_import_sc_df.append(sc_df.loc[to_append, :], sort=False)
    gsheet_import_sc_appended = gsheet_import_sc_appended.reset_index()
    print('New sheet Dataframe has (rows, columns): ' + str(gsheet_import_sc_appended.shape))
    
    return gsheet_import_sc_appended

def sc_gsheet_uploader(gsheet_import_sc_appended):
    gc = pygsheets.authorize(client_secret='client_secret.json')
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wsnBd3AHObl4gnUJ2MlwkusuXRoOsQpu2Kx6zGH8bVY/edit#gid=0')
    sh[0].clear('A2')
    sh[0].set_dataframe(gsheet_import_sc_appended, 'A2', copy_index=False, copy_head=False, extend=False, fit=True, escape_formulae=True, nan='')
    print('New data has been succesfully uploaded!')

In [74]:
# Full function run
sc_df = SC_formatter()
print('------------------')
gsheet_import_sc_appended = sc_gsheet_appender(sc_df)
print('------------------')
sc_gsheet_uploader(gsheet_import_sc_appended)

Successfully returned the MySQL query
Successfully formatted the MySQL query
Successfully joined to Food_Mapping.xslx
SC_Formatter completed! Shape is (621, 33)
------------------
Successfully returned Sams Club.
Sams Club Dataframe has (rows, columns): (621, 33)
Imported Gsheet Dataframe has (rows, columns): (1, 33)
Rows to append: 620
New sheet Dataframe has (rows, columns): (621, 33)
------------------
New data has been succesfully uploaded!


In [64]:
import pygsheets
gc = pygsheets.authorize(client_secret='client_secret.json')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wsnBd3AHObl4gnUJ2MlwkusuXRoOsQpu2Kx6zGH8bVY/edit#gid=0')
sh[0].clear('A2')
sh[0].set_dataframe(gsheet_import_1, 'A2', copy_index=False, copy_head=False, extend=False, fit=True, escape_formulae=True, nan='')

# BREAK

In [38]:
sc_df.to_excel('test3.xlsx')

In [ ]:
sc_df_test = sc_df.loc[:50, :]
print(sc_df_test.shape)
sc_df_test.head()
sc_df_test.to_excel('test.xlsx')

In [64]:
gsheet_import = gsheet_import_caselist('Sams Club')
gsheet_import.head()

Successfully returned Sams Club.


,Program,Tags,Notes,Weeks post,Start date,End date,Item number,Item description,Category,Subcategory,Category num,Subcat num,Freeosk clubs,Control clubs,Transactions,Audience,Traffic/club,Scans,Scans/club,Engagement,Conversion,Repeat,Immediate,A%,B%,C%,Freeosk lift,Control lift,Merchandising type,Placement type,Program2,Unique identifier,Food
0,FSC19006_247824_20190308_12.zip,High repeat; Low engagement; High C%; Low lift,Low lift and engagement as it's a niche sub ca...,12W,2019-03-08,2019-03-14,247824,FANCY FEAST VARIETY,PET SUPPLIES,Moist Cat Food,8,22,86,437,1031197,56668,11990.66,658.93,5.50%,2.30%,38.80%,37%,37%,11%,52%,1%,0%,Tier 1 - Sampled,Sample,FSC19006,FSC1900643532,No,Yes


In [123]:
sh[0].clear()
sh[0].set_dataframe(gsheet_import_1, 'A3', copy_index=False, copy_head=False, extend=False, fit=True, escape_formulae=True)

# Walmart

In [42]:
"""SELECT traceable.*, lift.Control_lift, lift.Freeosk_lift
FROM longitudinal.traceable as traceable
JOIN longitudinal.lift as lift ON traceable.program_name = lift.Program_name
WHERE traceable.Weeks_post = '12W';"""

SyntaxError: EOL while scanning string literal (<ipython-input-42-a717187f5524>, line 1)

In [73]:
import datetime
from datetime import date
current = datetime.datetime(2020,2,21)
current - datetime.timedelta(days=182)

datetime.datetime(2019, 8, 23, 0, 0)

In [ ]:
['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 'End date', 'Item number', 'Item description', 'Category', 'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 'Control clubs', 'Transactions', 'Scans', 'Audience', 'Engagement', 'Conversion', 'Repeat', 'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 'Merchandising type', 'Traffic/club', 'Scans/club', 'Program2', 'Unique identifier', 'Placement type', 'cat_description', 'Food', 'subcat_description', 'CATEGORY_NUMBER', 'SUB_CATEGORY_NUMBER']